##### Note: Running the whole notebook takes just over 3 minutes.

## Setup

In [1]:
# import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from api_keys import gkey

ModuleNotFoundError: No module named 'api_keys'

In [ ]:
# load in csv files
county_crime_csv = "Data Files/2010 CA County Crime Data.csv" 
county_officer_csv = "Data Files/2010 CA County LEO Data.csv"
city_crime_csv = "Data Files/2010 CA City Crime Data.csv"
city_officer_csv = "Data Files/2010 CA City LEO Data.csv"
population_data_csv = "Data Files/CA Census Population Data.csv"
county_area_csv = "Data Files/County Land Area.csv"

In [ ]:
# read the csv files to data frames
county_crime_df = pd.read_csv(county_crime_csv)
county_officers_df = pd.read_csv(county_officer_csv)
city_crime_df = pd.read_csv(city_crime_csv)
city_officers_df = pd.read_csv(city_officer_csv)
population_df = pd.read_csv(population_data_csv)
counties_df = pd.read_csv(county_area_csv)

## Clean the County Crime Data

In [ ]:
# drop extraneuos rows before the header row
county_crime_df = county_crime_df.drop([0,1,2,3]).reset_index(drop=True)

# define the headers, remove their row, and insert the headers as column names
headers = county_crime_df.iloc[0]
county_crime_df = county_crime_df[1:]
county_crime_df.columns = headers
county_crime_df = county_crime_df.reset_index(drop=True)

In [ ]:
# remove '\n' from column names and rename the 'county type' column header
county_crime_df.columns = county_crime_df.columns.str.strip().str.replace('\n','')
county_crime_df.rename(columns={'Metropolitan/Nonmetropolitan':'County Type'}, inplace=True)

In [ ]:
# fill the county type down to replace the NaN values and remove 'Counties' from the end of the county type
county_crime_df['County Type'].fillna(method='ffill', inplace=True)
county_crime_df['County Type'] = county_crime_df['County Type'].str.split().str.get(0)

In [ ]:
# remove Tulare County (because of incomplete data) and the last row with NaN values from a footnote
county_crime_df = county_crime_df.dropna().reset_index(drop=True)

In [ ]:
# moving the county name in front of the county type
county_crime_df.rename(columns={'County':'County old'}, inplace=True)
county_crime_df.insert(0, 'County', county_crime_df['County old'])

# remove the old 'County' column
del county_crime_df['County old']

# removing Arson column as it is not included in property crime totals
del county_crime_df['Arson']

In [ ]:
# change data type of the crime counts to integer
county_crime_df['Violent crime'] = county_crime_df['Violent crime'].str.replace(',', '').astype(int)
county_crime_df['Murder and nonnegligent manslaughter'] = county_crime_df['Murder and nonnegligent manslaughter'].str.replace(',', '').astype(int)
county_crime_df['Forcible rape'] = county_crime_df['Forcible rape'].str.replace(',', '').astype(int)
county_crime_df['Robbery'] = county_crime_df['Robbery'].str.replace(',', '').astype(int)
county_crime_df['Aggravated assault'] = county_crime_df['Aggravated assault'].str.replace(',', '').astype(int)
county_crime_df['Property crime'] = county_crime_df['Property crime'].str.replace(',', '').astype(int)
county_crime_df['Burglary'] = county_crime_df['Burglary'].str.replace(',', '').astype(int)
county_crime_df['Larceny-theft'] = county_crime_df['Larceny-theft'].str.replace(',', '').astype(int)
county_crime_df['Motor vehicle theft'] = county_crime_df['Motor vehicle theft'].str.replace(',', '').astype(int)

In [ ]:
# display the county crime data frame
county_crime_df.head()

## Clean the City Crime Data & Pull Corresponding Counties from Google Maps API

In [ ]:
# drop extraneuos rows before the header row
city_crime_df = city_crime_df.drop([0,1,2]).reset_index(drop=True)

# define the headers, remove their row, and insert the headers as column names
headers = city_crime_df.iloc[0]
city_crime_df = city_crime_df[1:]
city_crime_df.columns = headers
city_crime_df = city_crime_df.reset_index(drop=True)

In [ ]:
# clean column headers and delete extraneous columns
city_crime_df.columns = city_crime_df.columns.str.strip().str.replace('\n',' ')
del city_crime_df['Population']
del city_crime_df['Arson']

In [ ]:
# drop extra rows that have NaN values
city_crime_df = city_crime_df.dropna().reset_index(drop=True)

In [ ]:
# insert column to store county names
city_crime_df.insert(1, 'County', '')

In [ ]:
# base url for google maps api calls
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?key='

# loop through data frame, call the api for the city, and pull out the county into the 'County' column
for index, row in city_crime_df.iterrows():
    
    city = row['City']
    
    query_url = base_url + gkey + '&address=' + city + ',California'
    
    response = requests.get(query_url).json()
    
    if((response['results'][0]['address_components'][1]['long_name'].rsplit(' ', 1)[-1]) == 'County'):
        county = response['results'][0]['address_components'][1]['long_name']
    else:
        county = response['results'][0]['address_components'][2]['long_name']
   
    city_crime_df.loc[index, 'County'] = county.rsplit(' ', 1)[0]

In [ ]:
# change data type of the crime counts to integer
city_crime_df['Violent crime'] = city_crime_df['Violent crime'].str.replace(',', '').astype(int)
city_crime_df['Murder and nonnegligent manslaughter'] = city_crime_df['Murder and nonnegligent manslaughter'].str.replace(',', '').astype(int)
city_crime_df['Forcible rape'] = city_crime_df['Forcible rape'].str.replace(',', '').astype(int)
city_crime_df['Robbery'] = city_crime_df['Robbery'].str.replace(',', '').astype(int)
city_crime_df['Aggravated assault'] = city_crime_df['Aggravated assault'].str.replace(',', '').astype(int)
city_crime_df['Property crime'] = city_crime_df['Property crime'].str.replace(',', '').astype(int)
city_crime_df['Burglary'] = city_crime_df['Burglary'].str.replace(',', '').astype(int)
city_crime_df['Larceny- theft'] = city_crime_df['Larceny- theft'].str.replace(',', '').astype(int)
city_crime_df['Motor vehicle theft'] = city_crime_df['Motor vehicle theft'].str.replace(',', '').astype(int)

In [ ]:
# group the cities by county and sum the crime counts
grouped_city_crime_df = city_crime_df.groupby(['County']).sum()

In [ ]:
# remove Tulare County from the data frame
grouped_city_crime_df = grouped_city_crime_df.drop('Tulare').reset_index()

In [ ]:
# display the new data frame of city agency crimes grouped by county
grouped_city_crime_df.head()

## Clean the County Police Officer Data

In [ ]:
# drop extraneuos rows before the header row
county_officers_df = county_officers_df.drop([0,1,2]).reset_index(drop=True)

# define the headers, remove their row, and insert the headers as column names
headers = county_officers_df.iloc[0]
county_officers_df = county_officers_df[1:]
county_officers_df.columns = headers
county_officers_df = county_officers_df.reset_index(drop=True)

In [ ]:
# clean the column headers and delete extraneous columns
county_officers_df.columns = county_officers_df.columns.str.strip().str.replace('\n',' ')
del county_officers_df['Metropolitan/Nonmetropolitan']
del county_officers_df['Total law enforcement employees']
del county_officers_df['Total civilians']

In [ ]:
# remove Tulare County
county_officers_df = county_officers_df[county_officers_df['County'] != 'Tulare']
county_officers_df = county_officers_df.dropna().reset_index(drop=True)

In [ ]:
# change the data type of the officer count to integer
county_officers_df['Total officers'] = county_officers_df['Total officers'].str.replace(',', '').astype(int)

In [ ]:
# display the county officers data frame
county_officers_df.head()

## Clean the City Police Officer Data & Pull Corresponding Counties from Google Maps API

In [ ]:
# drop extraneuos rows before the header row
city_officers_df = city_officers_df.drop([0,1,2]).reset_index(drop=True)

# define the headers, remove their row, and insert the headers as column names
headers = city_officers_df.iloc[0]
city_officers_df = city_officers_df[1:]
city_officers_df.columns = headers
city_officers_df = city_officers_df.reset_index(drop=True)

In [ ]:
# clean column headers and delete extraneous columns
city_officers_df.columns = city_officers_df.columns.str.strip().str.replace('\n',' ')
del city_officers_df['Population']
del city_officers_df['Total law enforcement employees']
del city_officers_df['Total civilians']

In [ ]:
# drop extra rows that have NaN values
city_officers_df = city_officers_df.dropna().reset_index(drop=True)

city_officers_df.insert(1, 'County', '')

In [ ]:
# base url for google maps api calls
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?key='

# loop through data frame, call the api for the city, and pull out the county into the 'County' column

for index, row in city_officers_df.iterrows():
    
    city = row['City']
    
    query_url = base_url + gkey + '&address=' + city + ',California'
    
    response = requests.get(query_url).json()
    
    if((response['results'][0]['address_components'][1]['long_name'].rsplit(' ', 1)[-1]) == 'County'):
        county = response['results'][0]['address_components'][1]['long_name']
    else:
        county = response['results'][0]['address_components'][2]['long_name']
    
    city_officers_df.loc[index, 'County'] = county.rsplit(' ', 1)[0]

In [ ]:
# change data type of the officer count to integer
city_officers_df['Total officers'] = city_officers_df['Total officers'].str.replace(',', '').astype(int)

In [ ]:
# group the cities by county and sum the officer counts
grouped_city_officers_df = city_officers_df.groupby(['County']).sum().reset_index()

In [ ]:
# remove Tulare County 
grouped_city_officers_df = grouped_city_officers_df[grouped_city_officers_df['County'] != 'Tulare']

In [ ]:
# display the new data frame of city officers grouped by county
grouped_city_officers_df.head()

## Merge Crime Data Frames

In [ ]:
# create new data frame that will be used to merge crime data
total_crime_df = county_crime_df

In [ ]:
# merge county crime with the grouped city crime
total_crime_df = total_crime_df.merge(grouped_city_crime_df, on='County', how='outer', suffixes=('_county', '_grouped')).fillna(0)

In [ ]:
# add Metropolitan label to San Francisco and Solano, which weren't listed in the county crime data
total_crime_df['County Type'] = total_crime_df['County Type'].replace(0, 'Metropolitan')

In [ ]:
# combine each of the offense types into new total columns
total_crime_df['Violent Crime'] = (total_crime_df['Violent crime_county'] + total_crime_df['Violent crime_grouped']).astype(int)
total_crime_df['Murder and Nonnegligent Manslaughter'] = (total_crime_df['Murder and nonnegligent manslaughter_county'] + total_crime_df['Murder and nonnegligent manslaughter_grouped']).astype(int)
total_crime_df['Forcible Rape'] = (total_crime_df['Forcible rape_county'] + total_crime_df['Forcible rape_grouped']).astype(int)
total_crime_df['Robbery'] = (total_crime_df['Robbery_county'] + total_crime_df['Robbery_grouped']).astype(int)
total_crime_df['Aggravated Assault'] = (total_crime_df['Aggravated assault_county'] + total_crime_df['Aggravated assault_grouped']).astype(int)
total_crime_df['Property Crime'] = (total_crime_df['Property crime_county'] + total_crime_df['Property crime_grouped']).astype(int)
total_crime_df['Burglary'] = (total_crime_df['Burglary_county'] + total_crime_df['Burglary_grouped']).astype(int)
total_crime_df['Larceny - Theft'] = (total_crime_df['Larceny-theft'] + total_crime_df['Larceny- theft']).astype(int)
total_crime_df['Motor Vehicle Theft'] = (total_crime_df['Motor vehicle theft_county'] + total_crime_df['Motor vehicle theft_grouped']).astype(int)

In [ ]:
# delete columns we no longer need
del total_crime_df['Violent crime_county']
del total_crime_df['Murder and nonnegligent manslaughter_county']
del total_crime_df['Forcible rape_county']
del total_crime_df['Robbery_county']
del total_crime_df['Aggravated assault_county']
del total_crime_df['Property crime_county']
del total_crime_df['Burglary_county']
del total_crime_df['Larceny-theft']
del total_crime_df['Motor vehicle theft_county']
del total_crime_df['Violent crime_grouped']
del total_crime_df['Murder and nonnegligent manslaughter_grouped']
del total_crime_df['Forcible rape_grouped']
del total_crime_df['Robbery_grouped']
del total_crime_df['Aggravated assault_grouped']
del total_crime_df['Property crime_grouped']
del total_crime_df['Burglary_grouped']
del total_crime_df['Larceny- theft']
del total_crime_df['Motor vehicle theft_grouped']

In [ ]:
# display the total crime data frame
total_crime_df.head()

## Merge Police Officer Data Frames

In [ ]:
# create new dataframe for officer data to be merged into
total_officers_df = county_officers_df

In [ ]:
# merge county officer data with the grouped city officer data
total_officers_df = total_officers_df.merge(grouped_city_officers_df, on='County', how='outer', suffixes=('_county', '_grouped'))

In [ ]:
# replace NaN values with zero and add officer columns to get total
total_officers_df.fillna(0, inplace=True)
total_officers_df['Total Officers'] = (total_officers_df['Total officers_county'] + total_officers_df['Total officers_grouped']).astype(int)

In [ ]:
# delete columns we no longer need
del total_officers_df['Total officers_county'] 
del total_officers_df['Total officers_grouped']

In [ ]:
# display the total officers data frame
total_officers_df.head()

## Merge Police Officer Data into Crime Data Frame

In [ ]:
# merge total crime with total officers
crime_officer_data = total_crime_df.merge(total_officers_df, on='County')

In [ ]:
# display the crime and officer data frame
crime_officer_data.head()

## Clean County Population Data & Merge into the Crime & Officer Data Frame

In [ ]:
# drop extraneuos rows before the header row
population_df = population_df.drop([0,2,3]).reset_index(drop=True)

# define the headers, remove their row, and insert the headers as column names
headers = population_df.iloc[0]
population_df = population_df[1:]
population_df.columns = headers
population_df = population_df.reset_index(drop=True)

In [ ]:
# remove '\n' from column headers
population_df.columns = population_df.columns.str.strip().str.replace('\n','')

In [ ]:
# extract County Name, Population for 2010, and sumlvl
population_df = population_df[['County','April 1,2010','sumlvl']]

In [ ]:
# remove all rows that are not at the county level
population_df = population_df.loc[population_df['sumlvl'] == '050'].reset_index(drop=True)

In [ ]:
# remove the sumlvl column and rename the date of the population to 'Population'
del population_df['sumlvl']
population_df = population_df.rename(columns={'April 1,2010':'Population'})

In [ ]:
# remove Tulare County
population_df = population_df.loc[population_df['County'] != 'Tulare']
# remove Klamath County, NaN because it no longer exists
population_df = population_df.dropna().reset_index(drop=True)

In [ ]:
# change the data type of 'Population' into an integer
population_df['Population'] = population_df['Population'].str.replace(',', '').astype(int)

In [ ]:
# merge the population column into the crime and officer data frame
crime_officer_pop = crime_officer_data.merge(population_df, on='County')

In [ ]:
# display the crime, officer, and population data frame
crime_officer_pop.head()

## Clean County Land Area Data & Merge into the Crime, Officer, & Population Data Frame

In [ ]:
# limit county land area data down to California counties
ca_counties_df = counties_df.loc[(counties_df['STCOU'] > 6000) & (counties_df['STCOU'] < 6200)]
ca_counties_df = ca_counties_df.reset_index(drop=True)

In [ ]:
# extract the county name and land area for 2010
ca_counties_df = ca_counties_df[['Areaname','LND110210D']]

In [ ]:
# define the county from the areaname column
county = ca_counties_df['Areaname'].str.split(',',1, expand=True)

In [ ]:
# insert the county into a new column
ca_counties_df['County'] = county[0]

In [ ]:
# insert land area into a new column and format as an integer
ca_counties_df['Land Area'] = ca_counties_df['LND110210D'].astype(int)

In [ ]:
# delete old columns
del ca_counties_df['Areaname']
del ca_counties_df['LND110210D']

In [ ]:
# remove Tulare County
ca_counties_df = ca_counties_df.loc[ca_counties_df['County'] != 'Tulare'].reset_index(drop=True)

In [ ]:
# display the California counties land area data frame
ca_counties_df.head()

In [ ]:
# merge land area into crime, officer, population dataframe to make the output data frame
output_data = crime_officer_pop.merge(ca_counties_df, on='County')

In [ ]:
# sort by county type and county to move the two metropolitan counties from the bottom 
output_data = output_data.sort_values(by=['County Type','County']).reset_index(drop=True)

In [ ]:
# output the final data frame to a csv file
output_data.to_csv('Data Files/Output Data Clean.csv', encoding='utf-8', index=False)

In [ ]:
# display the final data frame
output_data.head()